# python3: 函数式编程

在python中function也是一个物件。既然是物件，代表着可以被assign。而且可以当作function的传入值。

In [1]:
def func1(a):
    print(a)

a = func1
a(10)

def func2(f):
    f(20)
    
func2(func1)

10
20


## map/reduce

map的概念就是对于每一个iterable的物件都做一次function上的事情。map(func, iterable) 回传一个iterator。

In [2]:
print(list(map(abs, [i for i in range(-10,0)])))
print(list(map(str, [i for i in range(-10,0)])))

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
['-10', '-9', '-8', '-7', '-6', '-5', '-4', '-3', '-2', '-1']


对于 reduce而言，其实就是：

reduce(f, [x1, x2, x3, x4]) = f(f(f(x1, x2), x3), x4)

In [3]:
from functools import reduce

def add (x,y):
    return x+y
reduce(add,[x for x in range(10)])

45

以下是一个string to int的代码，比较让我觉得惊艳的是{}[]的用法。前面{}代表一个dict，后面的[]代表从中取value。

In [4]:
def char2num(s):
    return {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}[s]

def fun(x, y):
    return x*10 + y

reduce(fun, map(char2num, '13579'))

13579

python内建filter()，和map()的概念类似，filter(func, iterable) return iterator。只是，这个func是return True/False，如果是True就保留这个元素，如果是False就将这个元素剔除。

In [5]:
def func_false(x):
    return False
def func_true(x):
    return True

print(list(filter(func_false, [x for x in range(10)])))
print(list(filter(func_true, [x for x in range(10)])))

[]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


python中直接利用sorted()函数就可以对list进行排序。并且可以吃 key = func，代表要如何比较大小的方式。 reverse = False，代表是否需要反向。

[官方文件](https://docs.python.org/3/library/functions.html#sorted)

In [6]:
a = [2,3,1,51,-25,63,-62,4,-21]
print(sorted(a))
print(sorted(a, key = abs))
print(sorted(a, key = abs, reverse=True))

[-62, -25, -21, 1, 2, 3, 4, 51, 63]
[1, 2, 3, 4, -21, -25, 51, -62, 63]
[63, -62, 51, -25, -21, 4, 3, 2, 1]


刚刚看到python的star operator \* 在这边稍微提一下。基本上会把一个iterable拆开放到函数的input里面。同样，也有\*\*这个，是为了能够让dict拆开，对对应的input输入参数。

In [7]:
def func(a,b):
    print(a+b)
    
l = [2,3]
func(*l)
d = {"a":2, "b":3}
func(**d)

5
5


In [8]:
max([1,2,3,4])

4

# 返回函数

可以将函数直接返回，同样的，在 `f()` 的时候才会重新运算。

In [9]:
def lazy_sum(*args):
    def sum():
        ax = 0
        for n in args:
            ax = ax + n
        return ax
    return sum

f = lazy_sum(1,2,3)
print("f:", f)
print("f():", f())

f: <function lazy_sum.<locals>.sum at 0x1046b3840>
f(): 6


每次的回传function都会是不同的函数，并且调用结果不互相影响。

In [10]:
f1 = lazy_sum(1,2,3,4)
f2 = lazy_sum(1,2,3,4)
f3 = lazy_sum(5,6,7,8)

print("f1 == f2:", f1==f2)
print("f1():", f1())
print("f2():", f2())
print("f3():", f3())

f1 == f2: False
f1(): 10
f2(): 10
f3(): 26


### 注意

** 返回函数不要引用任何循环变量，或者后续会发生变换的变量 **

In [11]:
def count():
    fs = []
    for i in range(1,4):
        def f():
            return i*i;
        fs.append(f)
    return fs
f1, f2, f3 = count()

print("f1():", f1())
print("f2():", f2())
print("f3():", f3())

f1(): 9
f2(): 9
f3(): 9


感觉上结果是1，4，9。但是，由于在return的时候函数并没有执行，当真正执行的时候** i **就变成了3，所以全部的结果都是9

## 闭包

是个大坑，还需要了解。到时候专门找资料来理解。

基本的想法好像是能够利用返回函数来带回一部分变数，以摆脱scope对变数的限制。

## 匿名函数

python的lambda比较简单，只能有一个表达式。
```python
lambda x: x*x
```
其实就是
```python
def f(x):
    return x*x
```

In [12]:
list(map(lambda x: x*x, [1,2,3,4,5,6,7,8,9]))

[1, 4, 9, 16, 25, 36, 49, 64, 81]

## 装饰器

先创建一个function。

In [13]:
def now():
    print("2017-8-31")

f = now
f()

2017-8-31


假设现在需要增强某些now的功能却不想改变内部的定义，这样就需要借助“装饰器”（Decorator)来实现。

本质上，decorator是一个**返回函数**的高阶函数。

如下就是一个装饰器的范例程式。

In [28]:
def log(func):
    def wrapper(*args, **kw):
        print("call {}()".format(func.__name__))
        return func(*args, **kw)
    return wrapper

借助@语法，将decorator加在func上。

In [29]:
@log
def now():
    print("2017-8-31")
    
now()

call now()
2017-8-31


如果没有return func的话看看会出现什么情况，并以此了解decorator。

In [25]:
def log_no_return_func(func):
    def wrapper(*args, **kw):
        print("call {}()".format(func.__name__))
    return wrapper

@log_no_return_func
def now():
    print("2017-8-31")
    
now()

call now()


竟然就没有执行now里面的内容。

将**@log**放在**now()**前面的意义是相当于：**now = log(now)**，所以如果wrapper不把now丢回来的话，now是无法执行的。而上面print出来的值是在log(now)的时候产生的。

由于**wrapper**吃的参数是\*args,\*\*kw所以什么函数都可以使用。但是，如果需要decorator吃参数就需要包三层。

In [30]:
def log(text):
    def decorator(func):
        def wrapper(*args, **kw):
            print("{} {}:".format(text, func.__name__))
            return func(*args, **kw)
        return wrapper
    return decorator

@log("excute")
def now():
    print("2017-8-31")

now()

excute now:
2017-8-31


对于三层嵌套的执行是： **now = log("execute")(now)** 

decorator由于包了一层wrapper，会导致改变函数的属性。比如\_\_name\_\_等属性会变成wrapper的。

In [31]:
now.__name__

'wrapper'

因为有functool.wrps可以帮忙做这些事情，就不需要写
```python
wrapper.__name__ = func.__name__
```
完整的decorator的写法是这样的

In [34]:
import functools

def log(func):
    @functools.wraps(func)
    def wrapper(*args, **kw):
        print("call {}:".format(func.__name__))
        return func(*args, **kw)
    return wrapper

如果带着参数

In [33]:
import functools

def log(text):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kw):
            print("call {}:".format(func.__name__))
            return func(*args, **kw)
        return wrapper
    return decorator

### 练习

请编写一个decorator，能在函数调用的前后打印出'begin call'和'end call'的日志。

In [46]:
import functools

def log(func):
    @functools.wraps(func)
    def wrapper(*args, **kw):
        print("begin call")
        func(*args, **kw)
        print("end call")
    return wrapper

@log
def fun():
    print("execute")
    
fun()

begin call
execute
end call


## 偏函数(Partial function)

在[Tensorflow的input function](https://www.tensorflow.org/get_started/input_fn)提到过如果input function要吃一些参数的时候，可是input function的定义是不能有参数的，就可以使用functools.partial来定住某些值，再将参数传入train。

这边利用**int()**来解释，int可以直接吃string，会直接转换成int。如果加上base这个参数，就可以基于这个base装换成10进制。

In [49]:
int_10 = int("12345")
int_8 = int("12345", base=8)
int_16 = int("12345", base=16)

print("int_10: {}".format(int_10))
print("int_8: {}".format(int_8))
print("int_16: {}".format(int_16))

int_10: 12345
int_8: 5349
int_16: 74565


可以使用partial来重新定义新的function。

In [55]:
import functools

int8 = functools.partial(int, base=8)
int_8 = int8("12345")

print("int_8: {}".format(int_8))

int_8: 5349
